In [2]:
# Smart Resume Screener - Collaborative Code
# Install required packages first:
!pip install streamlit PyPDF2 python-docx spacy scikit-learn plotly
# !python -m spacy download en_core_web_sm

import streamlit as st
import PyPDF2
import docx
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import plotly.express as px
from datetime import datetime
import io

# Initialize spaCy for NLP
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    st.error("Please download the spaCy model: python -m spacy download en_core_web_sm")
    st.stop()

class ResumeScreener:
    def __init__(self):
        self.skills_keywords = [
            'python', 'java', 'javascript', 'sql', 'html', 'css', 'react', 'node.js',
            'machine learning', 'deep learning', 'tensorflow', 'pytorch', 'aws',
            'docker', 'kubernetes', 'git', 'rest api', 'mongodb', 'mysql', 'postgresql',
            'data analysis', 'pandas', 'numpy', 'tableau', 'power bi', 'excel',
            'project management', 'agile', 'scrum', 'leadership', 'communication',
            'problem solving', 'teamwork', 'analytical skills'
        ]

    def extract_text_from_pdf(self, pdf_file):
        """Extract text from PDF file"""
        try:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
            return text
        except Exception as e:
            st.error(f"Error reading PDF: {str(e)}")
            return ""

    def extract_text_from_docx(self, docx_file):
        """Extract text from DOCX file"""
        try:
            doc = docx.Document(docx_file)
            text = ""
            for paragraph in doc.paragraphs:
                text += paragraph.text + "\n"
            return text
        except Exception as e:
            st.error(f"Error reading DOCX: {str(e)}")
            return ""

    def extract_skills(self, text):
        """Extract skills from resume text"""
        text_lower = text.lower()
        found_skills = []

        for skill in self.skills_keywords:
            if skill in text_lower:
                found_skills.append(skill)

        # Use spaCy for additional skill extraction
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ in ["ORG", "PRODUCT", "TECH"]:
                if ent.text.lower() not in found_skills and len(ent.text) > 2:
                    found_skills.append(ent.text.lower())

        return list(set(found_skills))

    def extract_experience(self, text):
        """Extract experience information"""
        # Look for years of experience patterns
        experience_patterns = [
            r'(\d+)\+?\s*years?',
            r'experience.*?(\d+)\+?\s*years?',
            r'(\d+)\+?\s*years?.*?experience'
        ]

        years = []
        for pattern in experience_patterns:
            matches = re.finditer(pattern, text.lower())
            for match in matches:
                try:
                    years.append(int(match.group(1)))
                except:
                    continue

        return max(years) if years else 0

    def extract_education(self, text):
        """Extract education information"""
        education_keywords = [
            'bachelor', 'master', 'phd', 'mba', 'bs', 'ms', 'ba', 'ma',
            'university', 'college', 'degree', 'graduated'
        ]

        text_lower = text.lower()
        education_info = []

        for keyword in education_keywords:
            if keyword in text_lower:
                # Extract context around the keyword
                start = max(0, text_lower.find(keyword) - 50)
                end = min(len(text_lower), text_lower.find(keyword) + 50)
                context = text[start:end].strip()
                education_info.append(context)

        return education_info

    def calculate_match_score(self, resume_text, job_description):
        """Calculate match score between resume and job description"""
        # TF-IDF based similarity
        vectorizer = TfidfVectorizer()
        corpus = [resume_text, job_description]
        tfidf_matrix = vectorizer.fit_transform(corpus)
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

        # Skills matching bonus
        resume_skills = self.extract_skills(resume_text)
        jd_skills = self.extract_skills(job_description)

        if jd_skills:
            skills_match_ratio = len(set(resume_skills) & set(jd_skills)) / len(jd_skills)
        else:
            skills_match_ratio = 0

        # Combine scores
        final_score = (similarity[0][0] * 0.6) + (skills_match_ratio * 0.4)

        return min(final_score * 10, 10), resume_skills, jd_skills

def main():
    st.set_page_config(page_title="Smart Resume Screener", page_icon="📄", layout="wide")

    st.title("🤖 Smart Resume Screener")
    st.markdown("Upload resumes and job descriptions to find the best matches!")

    # Initialize session state
    if 'resumes' not in st.session_state:
        st.session_state.resumes = []
    if 'job_description' not in st.session_state:
        st.session_state.job_description = ""

    # Initialize screener
    screener = ResumeScreener()

    # Sidebar for job description
    with st.sidebar:
        st.header("Job Description")
        job_description = st.text_area(
            "Paste the job description here:",
            height=300,
            value=st.session_state.job_description,
            help="Enter the complete job description for matching"
        )

        st.session_state.job_description = job_description

        st.header("Upload Resumes")
        uploaded_files = st.file_uploader(
            "Choose resume files (PDF/DOCX)",
            type=['pdf', 'docx'],
            accept_multiple_files=True,
            help="Upload multiple resumes in PDF or DOCX format"
        )

    # Main content area
    col1, col2 = st.columns([2, 1])

    with col1:
        if uploaded_files and job_description:
            st.subheader("📊 Resume Analysis Results")

            results = []
            for uploaded_file in uploaded_files:
                # Extract text based on file type
                if uploaded_file.type == "application/pdf":
                    text = screener.extract_text_from_pdf(uploaded_file)
                elif uploaded_file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
                    text = screener.extract_text_from_docx(uploaded_file)
                else:
                    text = ""

                if text:
                    # Analyze resume
                    skills = screener.extract_skills(text)
                    experience = screener.extract_experience(text)
                    education = screener.extract_education(text)
                    match_score, resume_skills, jd_skills = screener.calculate_match_score(text, job_description)

                    results.append({
                        'filename': uploaded_file.name,
                        'match_score': match_score,
                        'skills': skills,
                        'experience_years': experience,
                        'education_snippets': education,
                        'resume_skills': resume_skills,
                        'jd_skills': jd_skills,
                        'text': text[:500] + "..."  # Store first 500 chars for preview
                    })

            # Sort results by match score
            results.sort(key=lambda x: x['match_score'], reverse=True)

            # Display results
            for result in results:
                with st.expander(f"📄 {result['filename']} - Score: {result['match_score']:.1f}/10", expanded=False):
                    col_a, col_b, col_c = st.columns(3)

                    with col_a:
                        st.metric("Match Score", f"{result['match_score']:.1f}/10")

                    with col_b:
                        st.metric("Skills Found", len(result['skills']))

                    with col_c:
                        st.metric("Experience", f"{result['experience_years']} years")

                    # Skills analysis
                    st.subheader("🔧 Skills Analysis")
                    matched_skills = set(result['resume_skills']) & set(result['jd_skills'])
                    missing_skills = set(result['jd_skills']) - set(result['resume_skills'])

                    col_skills1, col_skills2 = st.columns(2)

                    with col_skills1:
                        st.write("✅ **Matched Skills:**")
                        if matched_skills:
                            for skill in list(matched_skills)[:10]:  # Show first 10
                                st.write(f"- {skill}")
                        else:
                            st.write("No matching skills found")

                    with col_skills2:
                        st.write("❌ **Missing Skills:**")
                        if missing_skills:
                            for skill in list(missing_skills)[:10]:  # Show first 10
                                st.write(f"- {skill}")
                        else:
                            st.write("All required skills matched!")

                    # Education preview
                    if result['education_snippets']:
                        st.subheader("🎓 Education Highlights")
                        for edu in result['education_snippets'][:3]:  # Show first 3
                            st.write(f"- {edu}")

                    # Resume preview
                    st.subheader("📋 Resume Preview")
                    st.text_area(
                        "First 500 characters:",
                        value=result['text'],
                        height=100,
                        key=f"preview_{result['filename']}"
                    )

            # Create summary dataframe
            if results:
                df_results = pd.DataFrame([{
                    'Filename': r['filename'],
                    'Match Score': r['match_score'],
                    'Skills Count': len(r['skills']),
                    'Experience (Years)': r['experience_years']
                } for r in results])

                st.subheader("📈 Summary Dashboard")

                # Visualization
                fig = px.bar(
                    df_results,
                    x='Filename',
                    y='Match Score',
                    title='Resume Match Scores',
                    color='Match Score',
                    color_continuous_scale='viridis'
                )
                st.plotly_chart(fig, use_container_width=True)

                # Download results
                csv = df_results.to_csv(index=False)
                st.download_button(
                    label="📥 Download Results as CSV",
                    data=csv,
                    file_name=f"resume_screening_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
                    mime="text/csv"
                )

        elif uploaded_files and not job_description:
            st.warning("⚠️ Please enter a job description to analyze resumes.")
        elif not uploaded_files and job_description:
            st.info("📁 Please upload resume files to analyze.")
        else:
            st.info("👈 Please upload resumes and enter a job description to get started.")

    with col2:
        st.subheader("ℹ️ How to Use")
        st.markdown("""
        1. **Enter Job Description** in the sidebar
        2. **Upload Resumes** (PDF/DOCX format)
        3. **View Analysis** including:
           - Match score (1-10)
           - Skills matching
           - Experience extraction
           - Education highlights

        **Features:**
        - 📊 Smart scoring algorithm
        - 🔧 Skills gap analysis
        - 📈 Visual comparisons
        - 📥 Export results
        """)

        if job_description:
            st.subheader("📋 Job Description Analysis")
            jd_skills = screener.extract_skills(job_description)
            st.write(f"**Required Skills Found:** {len(jd_skills)}")
            if jd_skills:
                for skill in jd_skills[:15]:  # Show first 15 skills
                    st.write(f"- {skill}")

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.7 MB/s eta 0:00:00


2025-10-15 17:51:00.789 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 17:51:00.791 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 17:51:00.994 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-10-15 17:51:00.995 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 17:51:00.997 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 17:51:00.998 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-15 17:51:01.000 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [3]:
# Enhanced version with LLM capabilities
# !pip install openai anthropic

import openai
import json

class EnhancedResumeScreener(ResumeScreener):
    def __init__(self, api_key=None):
        super().__init__()
        if api_key:
            openai.api_key = api_key

    def llm_enhanced_analysis(self, resume_text, job_description, resume_skills, jd_skills):
        """Use LLM for enhanced analysis and justification"""
        prompt = f"""
        Analyze the following resume against the job description and provide:
        1. Overall match score (1-10)
        2. Key strengths
        3. Areas for improvement
        4. Detailed justification

        Resume Skills: {', '.join(resume_skills)}
        Job Required Skills: {', '.join(jd_skills)}

        Resume Excerpt: {resume_text[:1500]}
        Job Description: {job_description[:1000]}

        Provide response in JSON format:
        {{
            "match_score": 0,
            "strengths": [],
            "improvements": [],
            "justification": ""
        }}
        """

        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are an expert HR analyst. Provide detailed, objective analysis."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3
            )

            analysis = json.loads(response.choices[0].message.content)
            return analysis
        except Exception as e:
            # Fallback to basic analysis
            return {
                "match_score": 0,
                "strengths": ["Basic skills matching available"],
                "improvements": ["Enable LLM for detailed analysis"],
                "justification": "LLM analysis not available"
            }

# Add this to your main function after the basic analysis
def enhanced_main():
    # Add LLM API key input in sidebar
    with st.sidebar:
        st.header("🤖 Advanced Features")
        api_key = st.text_input("OpenAI API Key (optional)", type="password")
        use_llm = st.checkbox("Enable AI Analysis")

    # In the analysis section, add:
    if use_llm and api_key:
        enhanced_screener = EnhancedResumeScreener(api_key)
        for result in results:
            llm_analysis = enhanced_screener.llm_enhanced_analysis(
                result['text'],
                job_description,
                result['resume_skills'],
                result['jd_skills']
            )
            result['llm_analysis'] = llm_analysis

            # Display LLM analysis in expander
            with st.expander("🤖 AI Analysis", expanded=False):
                st.metric("AI Match Score", f"{llm_analysis['match_score']}/10")
                st.write("**Strengths:**")
                for strength in llm_analysis['strengths']:
                    st.write(f"✅ {strength}")
                st.write("**Improvements:**")
                for improvement in llm_analysis['improvements']:
                    st.write(f"📝 {improvement}")
                st.write("**Justification:**")
                st.write(llm_analysis['justification'])

In [4]:
# Database support for storing results
# !pip install sqlalchemy

from sqlalchemy import create_engine, Column, String, Integer, Float, DateTime, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import sqlite3

Base = declarative_base()

class ResumeAnalysis(Base):
    __tablename__ = 'resume_analyses'

    id = Column(Integer, primary_key=True)
    filename = Column(String)
    match_score = Column(Float)
    skills_found = Column(Integer)
    experience_years = Column(Integer)
    analysis_date = Column(DateTime)
    job_description_hash = Column(String)
    resume_text_preview = Column(Text)

class DatabaseManager:
    def __init__(self, db_url="sqlite:///resume_analyses.db"):
        self.engine = create_engine(db_url)
        Base.metadata.create_all(self.engine)
        Session = sessionmaker(bind=self.engine)
        self.session = Session()

    def save_analysis(self, result, job_description):
        analysis = ResumeAnalysis(
            filename=result['filename'],
            match_score=result['match_score'],
            skills_found=len(result['skills']),
            experience_years=result['experience_years'],
            analysis_date=datetime.now(),
            job_description_hash=hash(job_description),
            resume_text_preview=result['text']
        )
        self.session.add(analysis)
        self.session.commit()

    def get_analysis_history(self):
        return self.session.query(ResumeAnalysis).all()

# Add to main function
def main_with_db():
    db_manager = DatabaseManager()

    # After analysis, save to database
    for result in results:
        db_manager.save_analysis(result, job_description)

    # Add history view in sidebar
    with st.sidebar:
        if st.button("View Analysis History"):
            history = db_manager.get_analysis_history()
            if history:
                st.subheader("📊 Analysis History")
                for analysis in history[-5:]:  # Last 5 analyses
                    st.write(f"{analysis.filename}: {analysis.match_score:.1f}")

/tmp/ipython-input-3156571352.py:9: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [5]:
# Enhanced skills matching with categories
class AdvancedResumeScreener(ResumeScreener):
    def __init__(self):
        super().__init__()
        self.skills_categories = {
            'programming': ['python', 'java', 'javascript', 'c++', 'c#', 'ruby', 'go', 'rust'],
            'web_dev': ['html', 'css', 'react', 'angular', 'vue', 'node.js', 'django', 'flask'],
            'databases': ['sql', 'mysql', 'postgresql', 'mongodb', 'redis', 'oracle'],
            'cloud': ['aws', 'azure', 'gcp', 'docker', 'kubernetes', 'terraform'],
            'data_science': ['pandas', 'numpy', 'tensorflow', 'pytorch', 'scikit-learn', 'r'],
            'tools': ['git', 'jenkins', 'jira', 'confluence', 'slack'],
            'soft_skills': ['leadership', 'communication', 'teamwork', 'problem solving']
        }

    def categorize_skills(self, skills):
        categorized = {category: [] for category in self.skills_categories.keys()}
        categorized['other'] = []

        for skill in skills:
            found = False
            for category, category_skills in self.skills_categories.items():
                if skill in category_skills:
                    categorized[category].append(skill)
                    found = True
                    break
            if not found:
                categorized['other'].append(skill)

        return categorized

    def calculate_advanced_match_score(self, resume_text, job_description):
        base_score, resume_skills, jd_skills = self.calculate_match_score(resume_text, job_description)

        # Category-based scoring
        resume_categories = self.categorize_skills(resume_skills)
        jd_categories = self.categorize_skills(jd_skills)

        category_bonus = 0
        for category in jd_categories:
            if jd_categories[category] and resume_categories[category]:
                category_match = len(set(resume_categories[category]) & set(jd_categories[category])) / len(jd_categories[category])
                category_bonus += category_match * 0.5  # Bonus for category coverage

        final_score = min(base_score + category_bonus, 10)
        return final_score, resume_categories, jd_categories

# Add to your display section
def display_skill_categories(resume_categories, jd_categories):
    st.subheader("📊 Skills by Category")

    for category in resume_categories:
        if resume_categories[category] or jd_categories[category]:
            col1, col2 = st.columns(2)

            with col1:
                st.write(f"**{category.title()}**")
                if resume_categories[category]:
                    for skill in resume_categories[category]:
                        st.write(f"✅ {skill}")
                else:
                    st.write("No skills found")

            with col2:
                st.write("**Required**")
                if jd_categories[category]:
                    for skill in jd_categories[category]:
                        if skill in resume_categories[category]:
                            st.write(f"✅ {skill}")
                        else:
                            st.write(f"❌ {skill}")

In [6]:
# requirements.txt for deployment
"""
streamlit==1.28.0
PyPDF2==3.0.1
python-docx==0.8.11
spacy==3.7.0
scikit-learn==1.3.0
plotly==5.15.0
pandas==2.1.0
openai==0.28.0
sqlalchemy==2.0.20
"""

# .streamlit/config.toml
"""
[theme]
primaryColor = "#FF4B4B"
backgroundColor = "#FFFFFF"
secondaryBackgroundColor = "#F0F2F6"
textColor = "#262730"
font = "sans serif"

[server]
headless = true
port = 8501
enableCORS = false
"""

# Dockerfile for container deployment
"""
FROM python:3.9-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install -r requirements.txt
RUN python -m spacy download en_core_web_sm

COPY . .

EXPOSE 8501

HEALTHCHECK CMD curl --fail http://localhost:8501/_stcore/health

ENTRYPOINT ["streamlit", "run", "resume_screener.py", "--server.port=8501", "--server.address=0.0.0.0"]
"""

'\nFROM python:3.9-slim\n\nWORKDIR /app\n\nCOPY requirements.txt .\nRUN pip install -r requirements.txt\nRUN python -m spacy download en_core_web_sm\n\nCOPY . .\n\nEXPOSE 8501\n\nHEALTHCHECK CMD curl --fail http://localhost:8501/_stcore/health\n\nENTRYPOINT ["streamlit", "run", "resume_screener.py", "--server.port=8501", "--server.address=0.0.0.0"]\n'

In [8]:
# Unit tests for the resume screener - Colab Compatible Version
import unittest
import sys
import io

class TestResumeScreener(unittest.TestCase):
    def setUp(self):
        self.screener = ResumeScreener()

    def test_skills_extraction(self):
        test_text = "I have experience with Python, JavaScript, and AWS."
        skills = self.screener.extract_skills(test_text)
        self.assertIn('python', skills)
        self.assertIn('javascript', skills)
        self.assertIn('aws', skills)
        print("✅ Skills extraction test passed")

    def test_experience_extraction(self):
        test_text = "I have 5 years of experience in software development."
        experience = self.screener.extract_experience(test_text)
        self.assertEqual(experience, 5)
        print("✅ Experience extraction test passed")

    def test_experience_extraction_multiple(self):
        test_text = "3 years as intern and 2 years as full-time developer with total 5 years experience."
        experience = self.screener.extract_experience(test_text)
        self.assertEqual(experience, 5)
        print("✅ Multiple experience extraction test passed")

    def test_education_extraction(self):
        test_text = "I have a Bachelor's degree in Computer Science from University of Test."
        education = self.screener.extract_education(test_text)
        self.assertTrue(len(education) > 0)
        self.assertTrue(any('bachelor' in edu.lower() for edu in education))
        print("✅ Education extraction test passed")

    def test_match_score_calculation(self):
        resume_text = "Expert in Python and SQL with database management skills."
        job_description = "We need Python developers with SQL experience for database projects."
        score, resume_skills, jd_skills = self.screener.calculate_match_score(resume_text, job_description)
        self.assertGreaterEqual(score, 0)
        self.assertLessEqual(score, 10)
        self.assertIn('python', resume_skills)
        self.assertIn('sql', resume_skills)
        print("✅ Match score calculation test passed")

    def test_pdf_text_extraction(self):
        # Create a simple PDF in memory for testing
        from PyPDF2 import PdfWriter, PdfReader
        import io

        # Create a PDF in memory
        pdf_writer = PdfWriter()
        pdf_writer.add_blank_page(width=200, height=200)

        # Add some text (this is simplified for testing)
        output = io.BytesIO()
        pdf_writer.write(output)
        output.seek(0)

        # Try to extract text (will be empty for blank page, but shouldn't crash)
        text = self.screener.extract_text_from_pdf(output)
        self.assertIsInstance(text, str)
        print("✅ PDF text extraction test passed")

    def test_skills_keywords_presence(self):
        self.assertGreater(len(self.screener.skills_keywords), 0)
        self.assertIn('python', self.screener.skills_keywords)
        self.assertIn('machine learning', self.screener.skills_keywords)
        print("✅ Skills keywords test passed")

# Function to run tests in Colab
def run_resume_screener_tests():
    """Run all tests and display results"""
    print("🧪 Running Resume Screener Tests...")
    print("=" * 50)

    # Create test suite
    test_suite = unittest.TestLoader().loadTestsFromTestCase(TestResumeScreener)

    # Capture test output
    test_runner = unittest.TextTestRunner(verbosity=2, stream=sys.stdout)

    # Run tests
    result = test_runner.run(test_suite)

    print("=" * 50)
    if result.wasSuccessful():
        print("🎉 All tests passed!")
    else:
        print(f"❌ {len(result.failures)} tests failed, {len(result.errors)} errors")

    return result

# Run the tests
run_resume_screener_tests()

🧪 Running Resume Screener Tests...
test_education_extraction (__main__.TestResumeScreener.test_education_extraction) ... ✅ Education extraction test passed
ok
test_experience_extraction (__main__.TestResumeScreener.test_experience_extraction) ... ✅ Experience extraction test passed
ok
test_experience_extraction_multiple (__main__.TestResumeScreener.test_experience_extraction_multiple) ... ✅ Multiple experience extraction test passed
ok
test_match_score_calculation (__main__.TestResumeScreener.test_match_score_calculation) ... ✅ Match score calculation test passed
ok
test_pdf_text_extraction (__main__.TestResumeScreener.test_pdf_text_extraction) ... ✅ PDF text extraction test passed
ok
test_skills_extraction (__main__.TestResumeScreener.test_skills_extraction) ... ✅ Skills extraction test passed
ok
test_skills_keywords_presence (__main__.TestResumeScreener.test_skills_keywords_presence) ... ✅ Skills keywords test passed
ok

---------------------------------------------------------------

<unittest.runner.TextTestResult run=7 errors=0 failures=0>

In [9]:
# For processing multiple resumes in batch
def batch_process_resumes(folder_path, job_description):
    """Process all resumes in a folder"""
    import os
    screener = ResumeScreener()
    results = []

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        if filename.endswith('.pdf'):
            with open(file_path, 'rb') as f:
                text = screener.extract_text_from_pdf(f)
        elif filename.endswith('.docx'):
            text = screener.extract_text_from_docx(file_path)
        else:
            continue

        if text:
            match_score, resume_skills, jd_skills = screener.calculate_match_score(text, job_description)
            results.append({
                'filename': filename,
                'match_score': match_score,
                'skills': resume_skills
            })

    return sorted(results, key=lambda x: x['match_score'], reverse=True)